# 뉴스 기사 크롤링 하기

- SK 하이닉스, 네이버, 삼성전자
- 참고 링크 : https://wonhwa.tistory.com/46#comment18426434
***

*참고 : **윈도우 환경에서 실행해야 정상적으로 돌아감**

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

In [ ]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search, start_page, end_page):
    if start_page == end_page:
        start_page = makePgNum(start_page)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        return url
    else:
        urls= []
        for i in range(start_page, end_page+1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        return urls

In [ ]:
def get_news(search, start_page, end_page) :
    search_urls = makeUrl(search, start_page, end_page)

    # selenium으로 navernews만 뽑아오기
    driver = driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.implicitly_wait(3)

    # selenium으로 검색 페이지 불러오기
    naver_urls=[]
    for i in search_urls:
        driver.get(i)
        time.sleep(2) #대기시간

        # 네이버 기사 눌러서 제목 및 본문 가져오기#
        # 네이버 기사가 있는 기사 css selector 모아오기
        a = driver.find_elements(By.CSS_SELECTOR,'a.info')

        # 위에서 생성한 css selector list 하나씩 클릭하여 본문 url얻기
        for i in a:
            i.click()

            # 현재탭에 접근
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(2)

            url = driver.current_url
            # 네이버 뉴스 url만 가져오기
            if "n.news.naver.com" in url:
                naver_urls.append(url)

            # 현재 탭 닫기
            driver.close()
            # 다시처음 탭으로 돌아가기
            driver.switch_to.window(driver.window_handles[0])

    # naver 기사 본문 및 제목 가져오기
    # ConnectionError방지
    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102" }

    news = []
    for i in naver_urls:
        original_html = requests.get(i,headers=headers)
        soup = BeautifulSoup(original_html.text, "html.parser")

        # 데이터 가져오기
        title = soup.select('meta[property="og:title"]')[0].get('content') #제목
        rink = i # 링크
        upload_time = soup.select('div.media_end_head_info_datestamp_bunch')[0]('span')[0].text # 작성 날짜

        news.append({
            'Title' : title,
            'Time' : upload_time,
            'Rink' : rink
        })

    return news

In [ ]:
# sk 뉴스 크롤링
sk_news = get_news('SK하이닉스', 1, 3)
print(sk_news)

[{'Title': '美 반도체 급락 여파… SK하이닉스·삼성전자 나란히 약세', 'Time': '2024.04.22. 오전 9:25', 'Rink': 'https://n.news.naver.com/mnews/article/417/0000997980?sid=101'}, {'Title': '"SK하이닉스, TSMC와 협력은 이제 시작일뿐…목표가↑"-메리츠', 'Time': '2024.04.22. 오전 8:55', 'Rink': 'https://n.news.naver.com/mnews/article/015/0004975403?sid=101'}, {'Title': "엔비디아 노리는 삼성, SK하이닉스 '맹추격' 나섰다", 'Time': '2024.04.22. 오전 8:48', 'Rink': 'https://n.news.naver.com/mnews/article/008/0005028411?sid=101'}, {'Title': 'SK하이닉스-TSMC 기술 동맹...“6세대 HBM 공동 개발”', 'Time': '2024.04.19. 오전 9:37', 'Rink': 'https://n.news.naver.com/mnews/article/023/0003829308?sid=105'}, {'Title': '삼성전자·SK하이닉스, 차세대 AI 반도체 선점 경쟁 격화', 'Time': '2024.04.21. 오후 7:56', 'Rink': 'https://n.news.naver.com/mnews/article/022/0003926137?sid=101'}, {'Title': '‘엔비디아 쇼크’에 삼성전자·SK하이닉스 등 반도체株↓ [증시이슈]', 'Time': '2024.04.22. 오전 9:56', 'Rink': 'https://n.news.naver.com/mnews/article/243/0000058766?sid=101'}, {'Title': "'킹달러'에 외인 떠난 SK하이닉스·한미반도체…엔비디아發 악재까지", 'Time': '2024.04.22. 오전 6:00', 'Rin

In [ ]:
# naver 뉴스 크롤링
naver_news = get_news('네이버', 1, 3)
print(naver_news)

[{'Title': '삼성카드, 네이버웹툰 카드에 ‘유미의 세포들’ 신규 디자인 추가', 'Time': '2024.04.22. 오전 9:34', 'Rink': 'https://n.news.naver.com/mnews/article/243/0000058761?sid=101'}, {'Title': "삼성카드, '네이버웹툰 삼성 iD 카드' 유미의 세포들 신규 디자인 추가", 'Time': '2024.04.22. 오전 9:17', 'Rink': 'https://n.news.naver.com/mnews/article/018/0005720904?sid=101'}, {'Title': '“웹툰의 영화·드라마화 권리, 작가에게 줘야”… 네이버웹툰 등 불공정약관 시정', 'Time': '2024.04.21. 오후 12:01', 'Rink': 'https://n.news.naver.com/mnews/article/366/0000987168?sid=101'}, {'Title': "박태준만화회사, 신작 '배달왕' 네이버웹툰에 연재", 'Time': '2024.04.22. 오전 9:56', 'Rink': 'https://n.news.naver.com/mnews/article/003/0012504310?sid=105'}, {'Title': '韓 AI 모델 없다던 스탠포드대…네이버에 "수정작업 진행 중" 답변', 'Time': '2024.04.21. 오후 7:23', 'Rink': 'https://n.news.naver.com/mnews/article/003/0012503855?sid=105'}, {'Title': '쿠팡 멤버십 폭탄 인상에 네이버·신세계·컬리 “우리는 무료”', 'Time': '2024.04.22. 오전 9:02', 'Rink': 'https://n.news.naver.com/mnews/article/037/0000034296?sid=101'}, {'Title': "네이버, C커머스·쿠팡 공세에 AI 추천·당일배송 '승부수'", 'Time': '2024.04.19. 오

In [ ]:
# 삼성전자 뉴스 크롤링
samsung_news = get_news('삼성전자', 1, 3)
print(samsung_news)

[{'Title': '삼성전자, 2024년형 스마트모니터 출시…"연결 강화"', 'Time': '2024.04.22. 오전 9:57', 'Rink': 'https://n.news.naver.com/mnews/article/003/0012504313?sid=101'}, {'Title': '"가전은 밀레" 고집하던 유럽인들 \'변심\'…삼성 \'1위\' 일냈다', 'Time': '2024.04.21. 오후 1:26', 'Rink': 'https://n.news.naver.com/mnews/article/015/0004975205?sid=101'}, {'Title': '삼성전자, 2024년형 스마트모니터 출시', 'Time': '2024.04.22. 오전 8:13', 'Rink': 'https://n.news.naver.com/mnews/article/082/0001266379?sid=101'}, {'Title': '삼성전자·SK하이닉스, 차세대 AI 반도체 선점 경쟁 격화', 'Time': '2024.04.21. 오후 7:56', 'Rink': 'https://n.news.naver.com/mnews/article/022/0003926137?sid=101'}, {'Title': '“삼성전자 주가, 상승 여력의 50% 못 미쳐”', 'Time': '2024.04.22. 오전 9:02', 'Rink': 'https://n.news.naver.com/mnews/article/037/0000034295?sid=101'}, {'Title': '美 반도체 급락 여파… SK하이닉스·삼성전자 나란히 약세', 'Time': '2024.04.22. 오전 9:25', 'Rink': 'https://n.news.naver.com/mnews/article/417/0000997980?sid=101'}, {'Title': '삼성전자, 에버랜드에 ‘갤럭시AI’ 체험공간 오픈', 'Time': '2024.04.21. 오전 8:38', 'Rink': 'https://n.news.naver.com

In [ ]:
import pandas as pd

sk_crawlering_df = pd.DataFrame(sk_news)
naver_crawlering_df = pd.DataFrame(naver_news)
samsung_crawlering_df = pd.DataFrame(samsung_news)

In [ ]:
display(sk_crawlering_df.head(3))
display(naver_crawlering_df.head(3))
display(samsung_crawlering_df.head(3))

NameError: name 'sk_crawlering_df' is not defined

### 데이터 전처리

In [ ]:
import datetime

def get_date(str_date) :
    str_date = str_date[:10]
    date = datetime.datetime.strptime(str_date, '%Y.%m.%d')
    return date

In [ ]:
sk_crawlering_df['Time'] = sk_crawlering_df['Time'].map(get_date)
samsung_crawlering_df['Time'] = samsung_crawlering_df['Time'].map(get_date)
naver_crawlering_df['Time'] = naver_crawlering_df['Time'].map(get_date)

In [ ]:
display(sk_crawlering_df.head(3))
display(samsung_crawlering_df.head(3))
display(naver_crawlering_df.head(3))

,Title,Time,Rink
0,美 반도체 급락 여파… SK하이닉스·삼성전자 나란히 약세,2024-04-22,https://n.news.naver.com/mnews/article/417/000...
1,"""SK하이닉스, TSMC와 협력은 이제 시작일뿐…목표가↑""-메리츠",2024-04-22,https://n.news.naver.com/mnews/article/015/000...
2,"엔비디아 노리는 삼성, SK하이닉스 '맹추격' 나섰다",2024-04-22,https://n.news.naver.com/mnews/article/008/000...


,Title,Time,Rink
0,"삼성전자, 2024년형 스마트모니터 출시…""연결 강화""",2024-04-22,https://n.news.naver.com/mnews/article/003/001...
1,"""가전은 밀레"" 고집하던 유럽인들 '변심'…삼성 '1위' 일냈다",2024-04-21,https://n.news.naver.com/mnews/article/015/000...
2,"삼성전자, 2024년형 스마트모니터 출시",2024-04-22,https://n.news.naver.com/mnews/article/082/000...


,Title,Time,Rink
0,"삼성카드, 네이버웹툰 카드에 ‘유미의 세포들’ 신규 디자인 추가",2024-04-22,https://n.news.naver.com/mnews/article/243/000...
1,"삼성카드, '네이버웹툰 삼성 iD 카드' 유미의 세포들 신규 디자인 추가",2024-04-22,https://n.news.naver.com/mnews/article/018/000...
2,"“웹툰의 영화·드라마화 권리, 작가에게 줘야”… 네이버웹툰 등 불공정약관 시정",2024-04-21,https://n.news.naver.com/mnews/article/366/000...


In [ ]:
sk_crawlering_df.to_csv('sk_news.csv', index=False)
samsung_crawlering_df.to_csv('samsung_news.csv', index=False)
naver_crawlering_df.to_csv('naver_news.csv', index=False)